In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 분석에 필요한 라이브러리
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from pprint import pprint
from scipy.stats import chisquare

#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras.applications import *
#from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Softmax
#AUTOTUNE = tf.data.AUTOTUNE

import warnings
warnings.filterwarnings('ignore')

import statsmodels.formula.api as smf

from math import sqrt
#import shap

from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from xgboost import plot_importance

import random
#import shap
#import skimage

In [ ]:
!pip install xgboost

In [ ]:
#seed
seed = 88 #42, 100, 82(XGboost), 88(Light GBM)

In [ ]:
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
#base_path = 'D:\projects\MIMIC3_EXTRACT_R'
base_path = '/content/drive/MyDrive/Colab Notebooks/data'
df = pd.read_csv(base_path + "/ventilation_dataset_6.csv")

In [ ]:
df
df.head(20)

In [ ]:
#불필요한 index 삭제
data1 = df.drop(df.columns[0], axis = 1)
data = data1.rename(columns = {'mbp24':'mbp','vt24':'vt','ve24':'ve', 'hr24':'hr','rr24':'rr', 'mean_pimax24':'pimax','pco24':'pco2'})
data.head()

In [ ]:
#성별을 여성(F)=1, 남성(M)=0 으로 변환
#data.gender = data.gender.apply(lambda x : mapping_dict_gender[x])
#after_mapping_gender
#after_mapping_gender = data['gender'].apply(lambda x : mapping_dict_gender[x])
#after_mapping_gender

mapping_dict_gender = {'F':1,'M':0}
data.gender = data.gender.apply(lambda x : mapping_dict_gender[x])

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data = data.dropna()
data = data.reset_index(drop = True)
data

In [ ]:
#data.to_csv(base_path + '/dataset6_dropna.csv',header = True, index = False)

In [ ]:
#target 과 X 를 분리함.
X = data.drop(columns="extubation_failure", axis=1)
y = data.extubation_failure
print(X.shape, y.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
#StandardScaler: 기본 스케일, 각 피처의 평균을 0, 표준편차를 1로 변환
#RobustScaler: 위와 유사하지만 평균 대신 중간값(median)과 일분위, 삼분위값(quartile)을 사용하여 이상치 영향을 최소화
#MinMaxScaler: 모든 피처의 최대치와 최소치가 각각 1, 0이 되도록 스케일 조정
#Normalizer: 피처(컬럼)이 아니라 row마다 정규화되며, 유클리드 거리가 1이 되도록 데이터를 조정하여 빠르게 학습할 수 있게 함 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
len(y_test.loc[y_test ==1])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
# set KFold CV
kf = KFold(random_state = seed,
           n_splits = 5,
           shuffle=True,
           )

In [ ]:
#Estimator 정하기
estimator = LogisticRegression()
estimator.get_params().keys()

In [ ]:
# GridSearch가 찾을 parameter를 정의
param_grid = {
  'penalty' : ['none', 'l2'],
  'C' : [0.01, 0.1, 1, 10, 100], #c가 작을수록 제약이 크다..?
  'class_weight' : [None, 'balanced'], 
}

In [ ]:
# define grid_search
grid_search = GridSearchCV(estimator = estimator,
                           param_grid = param_grid,
                           cv = kf,
                           n_jobs=-1,
                           verbose = 2)

# fit with(x_train, y_train)
grid_search.fit(X_train, y_train)

##{'C': 1, 'class_weight': None, 'penalty': 'l2'}

In [ ]:
# grid_search.best_params_

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from xgboost import plot_importance

#Estimator 정하기
estimator = XGBClassifier()
estimator.get_params().keys()

In [ ]:
# GridSearch가 찾을 parameter를 정의
param_grid = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [4, 5, 6, 7, 8],
    'max_features' : [0.7, 0.8, 0.9],
    'learning_rate' : [0.001, 0.005, 0.01,0.05, 0.1, 0.5,], #더 넓은 범위도 고려0.0001, 0.0005, 0.001,0.005,
    'reg_lambda' : [0.0005,0.001,0.005,0.01,0.05,0.1,0.5] #더 넓은 범위도 고려0.0001, 0.01, 0.05, 0.1, 0.5
}

In [ ]:
# define grid_search
grid_search = GridSearchCV(estimator = estimator,
                           param_grid = param_grid,
                           cv = kf,
                           n_jobs=-1,
                           verbose = 2)

# fit with(x_train, y_train)
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)
best_xgb = grid_search.best_estimator_
pred_xgb = best_xgb.predict(X_test)
tn, fp, fn, tp = metrics.confusion_matrix(y_test, pred_xgb).ravel()
sensitivity = tp/(tp+fn)
specificity = tn/(fp+tn)

print('precision  :','{0:0.3f}'.format(metrics.precision_score(y_test, pred_xgb)))
print('recall(Sensitivity)     :','{0:0.3f}'.format(metrics.recall_score(y_test, pred_xgb))) 
print('accuracy   :','{0:0.3f}'.format(metrics.accuracy_score(y_test, pred_xgb)))
print('sensitivity:', sensitivity)
print('specificity:', specificity)
print("F1 score for XGBoost : %.4f" % f1_score(y_test, pred_xgb))

##{'learning_rate': 0.1, 'max_depth': 7, 'max_features': 0.8, 'n_estimators': 50, 'reg_lambda': 0.001}

In [1]:
import pickle

ModuleNotFoundError: No module named 'xgboost'

In [6]:
#pip install xgboost

     ---------------------------------------- 70.9/70.9 MB 7.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from xgboost.sklearn import XGBClassifier
import os
import numpy as np
import pandas as pd

In [15]:
#Estimator 정하기
clf =XGBClassifier('max_depth'== 5, 'max_features'== 0.7, 'n_estimators'== 200, 'reg_lambda'== 0.05)
#SAVE MODEL
with open("extfail_xgb_model.pickle", "wb") as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)


c:\Python\Python39\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective, use_label_encoder` as keyword args.
  warnings.warn(msg, FutureWarning)
c:\Python\Python39\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [ ]:


#target 과 X 를 분리함.
X = data.drop(columns="extubation_failure", axis=1)
y = data.extubation_failure

In [12]:
import pickle

In [ ]:
#SAVE MODEL
with open("extfail_xgb_model.pickle", "wb") as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

In [14]:
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
clf2.predict(X[0:1])


NameError: name 'X' is not defined

In [ ]:
from joblib import dump, load
dump(clf, 'filename.joblib')